# 🏗 Groundwork 🪚

In [6]:
from pandas import ExcelWriter, DataFrame, ExcelFile, merge, concat, read_excel
from openpyxl import load_workbook
from os import system, remove
from os.path import isfile
from glob import glob

xlsx = 'unite.xlsx'
chief = 'chief'
monthly_finance = [ f'M{n}' for n in range(1,10) ]

# 👷🏾‍♂️ Functional 👨🏽‍🏭

In [7]:
def newfile(name:str): 
    if isfile(xlsx): remove(xlsx)
    ExcelWriter(name).save()

def capacity(typefile:str)->list: return sorted([ file for file in glob(f'*.{typefile}') if '~$' not in file ])

def fill(file:str, namesheet:str, content:list, columns:list=None, mode:str='w', index:bool=False, header:bool=True):
    with ExcelWriter(file, mode=mode, engine="openpyxl", if_sheet_exists=(None if mode=='w' else 'overlay')) as writer:  
        content.to_excel(writer, sheet_name=namesheet, index=index, header=header, columns=columns)
        
def unitexls(fxls:list):
    list_sheet=[]
    for fxl in fxls:
        ef = ExcelFile(fxl)
        print('\n', fxl, ":", end=' ')
        for sheet in ef.sheet_names:
            list_sheet.append(sheet) if sheet not in list_sheet else list_sheet.append(sheet+'_'+fxl.replace('.xlsx',''))
            fill(content=read_excel(ef, sheet_name=sheet), file=xlsx, namesheet=list_sheet[-1], mode='a', header=True)
            print(list_sheet[-1], end=' ')

def rename(file:str, sheet:str, oldnamesheet:str='Sheet1'):
    fxl = load_workbook(file)
    fxl[oldnamesheet].title = sheet
    fxl.save(file)

def check(file:str, message:str='\nResult of checking:'):
    fxl = load_workbook(file)
    sheets = fxl.sheetnames
    print(message, sheets, end='\n\n')

def scan(file:str, namesheet:str, start:str='A', upto:str='A1') -> DataFrame:
    tabl = []
    fxl = load_workbook(file)
    cells = fxl[namesheet][start+'1':upto]
    columns = [ cells[0][c].value for c in range(len(cells[0])) ]
    for cell in cells[1:]: tabl.append([ cell[c].value for c in range(len(cell)) ])
    return DataFrame( tabl, columns=columns, index=list(range(1,int(upto[1:]))) )

def connect(dfs:DataFrame) -> DataFrame:
    for df in dfs[1:]: dfs[0] = dfs[0].join(df,how='right')
    return dfs[0]

def add2matrix(matrix:list, index:int, value:list): matrix[index]=matrix[index]+value

def column2rows(df:DataFrame, column:int) -> list:
    D=[[]]*column
    enum = -1
    for k,v in df.values: 
        if int(k)==1: enum+=1
        add2matrix(D,enum,[v])
    return D

def enable(file:str, turnoff:str="Microsoft Excel", afterseconds:int=60): system(f'''open {file}; sleep {afterseconds};   killall "{turnoff}" || taskkill /f /im {turnoff.split()[-1]}.exe /t''') # mac || win

# 🫡 Screenplay 📝

In [9]:
newfile(xlsx)
fxls = capacity('xlsx')
fxls.remove(xlsx)
unitexls(fxls)
rename(xlsx, chief)
check(xlsx)

# we work in file payment
dfme = scan(file=xlsx, namesheet='payment', upto='A87')
dfmp = scan(file=xlsx, namesheet='payment', upto='B87', start='B')
df = concat([ connect( [dfme,dfmp] ) ], ignore_index=True).drop_duplicates(ignore_index=True)

# we work in file packet
dfp = scan(file=xlsx, namesheet='packet', upto='B6')
dfp = DataFrame([ nm for d in df['packet'] for id,nm in dict(dfp.values).items() if d==id ], columns=['packet_name'])
df = connect( [df,dfp] )

# we work in file ID
dfef = scan(file=xlsx, namesheet='ID', upto='B24')
dfee = scan(file=xlsx, namesheet='ID', upto='C24', start='C')
dfee.rename(columns={'ID':'id'}, inplace = True)
dfen = scan(file=xlsx, namesheet='ID', upto='D24', start='D')
dfef = connect( [dfee,dfen,dfef] )
df = merge(df,dfef,on='id').drop_duplicates(subset=['id','packet'],ignore_index=True)

# add cash costs from payment file
dfmm = scan(file=xlsx, namesheet='payment', upto='D87', start='C')
dfmm = DataFrame( column2rows(dfmm,10), columns=monthly_finance )
df = connect( [df,dfmm] )

# write everything in xlsx file and rejoice
print(df, end='\n\n')
fill(xlsx, chief, df, mode='a')
enable(xlsx)


 ID.xlsx : ID 
 packet.xlsx : packet 
 payment.xlsx : payment 
Result of checking: ['chief', 'ID', 'packet', 'payment']

    id  packet           packet_name Назва закладу Область реєстрації  \
0  109       3   Хірургічна допомога             w             М.КИЇВ   
1  109       4  Стаціонарна допомога             w             М.КИЇВ   
2  105       3   Хірургічна допомога             v             М.КИЇВ   
3  105       4  Стаціонарна допомога             v             М.КИЇВ   
4   94       3   Хірургічна допомога             l   ДНІПРОПЕТРОВСЬКА   
5   94       4  Стаціонарна допомога             l   ДНІПРОПЕТРОВСЬКА   
6  132       3   Хірургічна допомога             t         РІВНЕНСЬКА   
7  132       4  Стаціонарна допомога             t         РІВНЕНСЬКА   
8   81       3   Хірургічна допомога             s         ПОЛТАВСЬКА   
9   81       4  Стаціонарна допомога             s         ПОЛТАВСЬКА   

  Форма власності       M1        M2        M3        M4        M5        